1. 문서의 내용을 읽는다.
2. 문서를 쪼갠다
3. 임베딩 -> 벡터 데이터베이스에 저장
4. 질문이 있을때, 벡터 데이스베이스에서 유사도 검색
5. 유사도 검색으로 가져온 문서를 LLM에 질문과 함께 전달

# 1. 문서의 내용을 읽는다.

In [ ]:
%pip install pdfplumber

In [10]:
import pdfplumber

text = ''
with pdfplumber.open('text.pdf') as pdf:
    for page in pdf.pages:
        text += page.extract_text()



In [ ]:
len(text)

# 2. 문서를 쪼갠다

In [ ]:
%pip install tiktoken

In [27]:
import tiktoken

def split_text(text, chunk_size):
    encoder = tiktoken.encoding_for_model("gpt-4o")
    encoding = encoder.encode(text)
    token_count = len(encoding) # 출력 161015 # gpt-4o의 max 토큰 수는 128000이기 떄문에 쪼개는게 중요하다.

    text_list = []
    for i in range(0, token_count, chunk_size):
        chunk = encoding[i: i+chunk_size]
        decoded = encoder.decode(chunk)
        text_list.append(decoded)
    return text_list

In [28]:
chunk_list = split_text(text, 1500)

# 3. 임베딩 -> 벡터 데이터베이스에 저장

In [ ]:
%pip install chromadb

In [17]:
import chromadb

chroma_client = chromadb.Client()

In [19]:
collection_name = 'tax_collection'
tax_collection = chroma_client.create_collection(collection_name)

In [23]:
import os

from dotenv import load_dotenv
from chromadb.utils.embedding_functions import OpenAIEmbeddingFunction

# 동작하지 않음
#load_dotenv()

openai_api_key = os.getenv('OPENAI_API_KEY')
openai_embedding = OpenAIEmbeddingFunction(api_key=openai_api_key, model_name='text-embedding-3-large')


In [25]:
tax_collection = chroma_client.get_or_create_collection(collection_name, embedding_function=openai_embedding)

In [30]:
id_list = []
for index in range(len(chunk_list)):
    id_list.append(f'{index}')

In [31]:
tax_collection.add(documents=chunk_list, ids=id_list)

# 4. 질문이 있을때, 벡터 데이스베이스에서 유사도 검색

In [37]:
query = '연봉 3000만원인 직장인의 소득세는 얼마인가요?'
retireved_pdf = tax_collection.query(query_texts=query, n_results=3)

In [38]:
retireved_pdf['documents'][0]

['령으로 정하는 특수관계에 있는 자가 받는 보상금은 제외한다.\n2) 대학의 교직원 또는 대학과 고용관계가 있는 학생이 소속 대학에 설치된 「산업교육진흥 및 산학연\n협력촉진에 관한 법률」 제25조에 따른 산학협력단(이하 이 조에서 “산학협력단”이라 한다)으로부터 같은 법\n제32조제1항제4호에 따라 받는 보상금\n저. 대통령령으로 정하는 복리후생적 성질의 급여\n처. 제20조제1항제6호에 따른 소득 중 다음의 요건을 모두 충족하는 소득으로서 대통령령으로 정하는 금액\n이하의 금액\n1) 임원 또는 종업원(이하 이 조, 제20조 및 제164조의5에서 “임원등”이라 한다) 본인이 소비하는 것을\n목적으로 제공받거나 지원을 받아 구입한 재화 또는 용역으로서 대통령령으로 정하는 기간 동안 재\n판매가 허용되지 아니할 것\n2) 해당 재화 또는 용역의 제공과 관련하여 모든 임원등에게 공통으로 적용되는 기준이 있을 것\n4. 연금소득 중 다음 각 목의 어느 하나에 해당하는 소득\n가. 「국민연금법」, 「공무원연금법」 또는 「공무원 재해보상법」, 「군인연금법」 또는 「군인 재해보상법\n」, 「사립학교교직원 연금법」, 「별정우체국법」 또는 「국민연금과 직역연금의 연계에 관한 법률」(이하\n“공적연금 관련법”이라 한다)에 따라 받는 유족연금ㆍ퇴직유족연금ㆍ퇴역유족연금ㆍ장해유족연금ㆍ상이\n유족연금ㆍ순직유족연금ㆍ직무상유족연금ㆍ위험직무순직유족연금, 장애연금, 장해연금ㆍ비공무상 장해연\n금ㆍ비직무상 장해연금, 상이연금(傷痍年金), 연계노령유족연금 또는 연계퇴직유족연금\n나. 삭제<2013. 1. 1.>\n다. 「산업재해보상보험법」에 따라 받는 각종 연금\n라. 「국군포로의 송환 및 대우 등에 관한 법률」에 따른 국군포로가 받는 연금\n마. 삭제<2013. 1. 1.>\n5. 기타소득 중 다음 각 목의 어느 하나에 해당하는 소득\n가. 「국가유공자 등 예우 및 지원에 관한 법률」 또는 「보훈보상대상자 지원에 관한 법률」에 따라 받는 보\n훈급여금ㆍ학습보조비 및 「북한이탈주민의 보호 

# 5. 유사도 검색으로 가져온 문서를 LLM에 질문과 함께 전달

In [39]:
%pip install openai


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [40]:
from openai import OpenAI

client = OpenAI()

# message에 history를 추가해서 질문을 할 수 있음
response = client.chat.completions.create(
    model='gpt-4o',
    messages=[
        {'role': 'system', 'content': f'당신은 한국의 소득세 전문가 입니다. 아래 내용을 참고해서 사용자의 질문에 답변해주세요. {retireved_pdf["documents"][0]}'},
        {'role': 'user', 'content': query},
    ],
)


In [41]:
response.choices[0].message.content

ChatCompletion(id='chatcmpl-BEypDIU1LPiZ4kbVQdB4cfrCZ54Sg', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='연봉 3,000만원인 직장인의 소득세를 계산하기 위해서는 우선 소득세율을 적용하여야 하며, 근로소득공제 및 기본공제를 고려해야 합니다. \n\n2023년 기준, 대한민국의 근로소득세 계산은 아래 단계로 이루어집니다: \n\n1. **근로소득공제**: 연봉에 따라 일정 금액이 공제됩니다. 연봉 3,000만원의 경우, 근로소득공제 금액은 다음과 같습니다:\n   - 500만원 이하: 전액 공제\n   - 500만원 초과 1,500만원 이하: 500만원 +\n   - 1,500만원 초과 4,500만원 이하: (연봉 - 1,500만원) × 10% + 1,000만원\n\n   구체적으로 계산하면, 3,000만원 - 1,500만원 = 1,500만원이 10%로 공제됩니다. 즉, 150만원 공제가 추가되므로 총 공제는 1,000만원 + 150만원 = 1,150만원입니다.\n\n2. **과세표준 계산**: 연봉에서 근로소득공제를 뺀 금액\n   - 과세표준 = 3,000만원 - 1,150만원 = 1,850만원\n\n3. **소득세율 적용**: 과세표준에 따라 소득세율을 적용합니다. \n   - 과세표준 1,200만원 초과 4,600만원 이하: 15%\n\n   과세표준에 대한 소득세는 (1,850만원 - 1,200만원) × 15% + 72만원(기본세액) = 9,750,000 × 0.15 + 720,000 = 1,462,500원이 됩니다.\n\n4. **세액공제**: 기본적으로 부양가족, 의료비, 교육비 등 여러 세액공제가 있고 이는 개인의 상황에 따라 다르게 적용될 수 있습니다. 일반적으로 적용되는 세액공제가 없다고 가정했을 때를 기초로 설명합니다.\n\n이를 통해 최종적인 소득세를 계산하게 